# SAM 모델 및 파라미터 다운로드

In [ ]:
!pip install segment_anything
!wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth
!pip install ultralytics

# 좌측 50 픽셀(줄자 부분) 제거

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import glob

# 1. 이미지 폴더 경로 설정
image_folder = "/content/drive/MyDrive/2nd core sample dataset/raw image/"

# 'cropped' 하위 폴더 경로 설정 및 생성
output_subfolder = os.path.join(image_folder, "cropped")
os.makedirs(output_subfolder, exist_ok=True)

# 2. 이미지 파일 목록 불러오기
# glob.glob()을 사용하여 해당 폴더 내의 모든 .tif 파일을 찾습니다.
file_paths = glob.glob(os.path.join(image_folder, "*.tif"))

# 이미지 파일이 하나도 없을 경우를 대비하여 확인
if not file_paths:
    print(f"지정된 경로에 이미지 파일이 없습니다: {image_folder}")
else:
    print(f"{len(file_paths)}개의 이미지 파일을 찾았습니다. 처리를 시작합니다.")

    # 3. 파일 목록을 순회하며 이미지 처리
    for img_path in file_paths:
        # 이미지 불러오기
        image = cv2.imread(img_path)

        # 이미지가 제대로 불러와졌는지 확인
        if image is None:
            print(f"이미지를 불러올 수 없습니다. 경로를 확인해주세요: {img_path}")
            continue  # 다음 이미지로 넘어갑니다.

        # BGR을 RGB로 변환 (시각화용)
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # 4. 이미지의 좌측 50픽셀 잘라내기
        cropped_image = image[:, 50:]
        cropped_image_rgb = image_rgb[:, 50:]

        # 5. 저장할 파일 이름 및 경로 설정
        base_name = os.path.splitext(os.path.basename(img_path))[0]
        output_filename = f"{base_name}_left50_cropped.tif"

        # ***여기서 output_subfolder 변수를 사용하여 경로를 올바르게 설정합니다.***
        output_path = os.path.join(output_subfolder, output_filename)

        # 6. 이미지 저장
        cv2.imwrite(output_path, cropped_image)
        print(f"좌측 50픽셀이 잘라낸 이미지가 '{output_path}'로 저장되었습니다.")

        # 7. 결과 시각화 (선택 사항)
        # plt.figure(figsize=(10, 5))

        # plt.subplot(1, 2, 1)
        # plt.title("Original Image")
        # plt.imshow(image_rgb)
        # plt.axis("off")

        # plt.subplot(1, 2, 2)
        # plt.title("Cut 50 pixels in Left")
        # plt.imshow(cropped_image_rgb)
        # plt.axis("off")

        # plt.tight_layout()
        # plt.show()

# SLIC -> SAM 영역 분할 코드

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os
import torch
import torch.nn as nn
from segment_anything import sam_model_registry, SamPredictor
from skimage.segmentation import slic
import timm

# -----------------
# 1.1 모델 및 예측기 로드
# -----------------
def load_sam_predictor(sam_checkpoint="sam_vit_h_4b8939.pth"):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    sam_model = sam_model_registry["vit_h"](checkpoint=sam_checkpoint).to(device)
    sam_predictor = SamPredictor(sam_model)
    return sam_predictor, device

# -----------------
# 1.2 이미지에 패딩 적용 함수
# -----------------
def pad_image(image_np, stride=32):
    h, w, _ = image_np.shape
    h_pad = (stride - h % stride) % stride
    w_pad = (stride - w % stride) % stride
    padded_image_np = np.pad(image_np, ((0, h_pad), (0, w_pad), (0, 0)), mode='constant', constant_values=0)
    return padded_image_np, h_pad, w_pad

# ----------------------------------------------------
# 1.3 SLIC 기반으로 SAM 프롬프트 생성 및 마스크 생성 (중심점 프롬프트+멀티마스크로 수정)
# ----------------------------------------------------
def create_pseudo_labels_with_slic(image_dir, output_dir):
    sam_predictor, device = load_sam_predictor()
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for img_file in os.listdir(image_dir):
        if not img_file.endswith(('.png', '.jpg', '.tif')):
            continue

        img_path = os.path.join(image_dir, img_file)
        base_filename = os.path.splitext(img_file)[0]

        original_image = Image.open(img_path).convert("RGB")
        original_image_np = np.array(original_image)

        # SLIC 파라미터는 현재 설정(n_segments=100, compactness=5)을 유지합니다.
        segments_slic = slic(original_image_np, n_segments=250, compactness=5, sigma=1, start_label=1)

        num_slic_segments = segments_slic.max()

        padded_image_np, _, _ = pad_image(original_image_np)
        sam_predictor.set_image(padded_image_np)

        print(f"'{img_file}'에 대해 {num_slic_segments}개의 SLIC 중심점을 다중 마스크 프롬프트로 사용합니다.")

        original_h, original_w, _ = original_image_np.shape
        merged_labels = np.zeros(original_image_np.shape[:2], dtype=np.uint8)
        current_label_id = 1

        for i in range(1, num_slic_segments + 1):
            mask_slic = (segments_slic == i)

            y_coords, x_coords = np.where(mask_slic)
            if len(y_coords) == 0:
                continue

            center_y = int(y_coords.mean())
            center_x = int(x_coords.mean())
            point_coords = np.array([[center_x, center_y]])
            point_labels = np.array([1])

            # <--- 수정된 부분: multimask_output=True로 설정하여 여러 마스크를 받습니다 --->
            masks, _, _ = sam_predictor.predict(
                point_coords=point_coords,
                point_labels=point_labels,
                box=None,
                multimask_output=True, # 여기를 True로 변경
            )
            # <--- 수정된 부분 끝 --->

            # SAM이 생성한 여러 마스크를 순차적으로 병합
            for mask in masks:
                resized_mask = mask[:original_h, :original_w]
                unassigned_area = (resized_mask > 0) & (merged_labels == 0)
                if np.any(unassigned_area):
                    merged_labels[unassigned_area] = current_label_id
                    current_label_id += 1

        pseudo_label_path = os.path.join(output_dir, f"{base_filename}_merged.npy")
        np.save(pseudo_label_path, merged_labels)
        print(f"'{img_file}'에 대한 SLIC+SAM 기반 가상 정답 맵 저장 완료. 레이블 수: {current_label_id - 1}")

        del original_image, original_image_np, padded_image_np, segments_slic, masks, merged_labels
        torch.cuda.empty_cache()

    print("모든 이미지에 대한 SLIC+SAM 가상 정답 마스크 생성 완료.")

# ----------------------------------------------------
# 새로운 함수: 모든 이미지-레이블 쌍을 나란히 시각화
# ----------------------------------------------------
def visualize_all_side_by_side(image_dir, pseudo_label_dir):
    """
    지정된 폴더의 모든 원본 이미지와 해당 가상 정답 마스크를 나란히 시각화합니다.
    """
    image_files = [f for f in os.listdir(image_dir) if f.endswith(('.png', '.jpg', '.tif'))]

    if not image_files:
        print("이미지 폴더에 시각화할 파일이 없습니다.")
        return

    high_contrast_colors = [
        (0, 0, 0),        # 배경
        (255, 0, 0),      # 밝은 빨간색
        (0, 255, 0),      # 밝은 녹색
        (0, 0, 255),      # 밝은 파란색
        (255, 255, 0),    # 노란색
        (255, 0, 255),    # 자홍색
        (0, 255, 255),    # 청록색
        (128, 0, 255),    # 밝은 보라색
        (255, 128, 0),    # 주황색
        (0, 128, 0),      # 짙은 녹색
        (128, 128, 128),  # 회색
        (255, 192, 203)   # 분홍색
    ]

    for img_file in image_files:
        base_filename = os.path.splitext(img_file)[0]
        pseudo_label_path = os.path.join(pseudo_label_dir, f"{base_filename}_merged.npy")

        if not os.path.exists(pseudo_label_path):
            print(f"경고: {img_file}에 대한 가상 정답 파일을 찾을 수 없습니다. 건너뜁니다.")
            continue

        original_image = Image.open(os.path.join(image_dir, img_file)).convert("RGB")
        pseudo_label = np.load(pseudo_label_path)

        num_classes = np.max(pseudo_label) + 1

        colored_mask = np.zeros((*pseudo_label.shape, 3), dtype=np.uint8)
        for class_id in range(num_classes):
            color = high_contrast_colors[class_id % len(high_contrast_colors)]
            colored_mask[(pseudo_label == class_id)] = color

        original_np = np.array(original_image)
        blended_image_np = (original_np * 0.4 + colored_mask * 0.6).astype(np.uint8)

        # 2개의 서브플롯을 생성하여 나란히 시각화
        fig, axes = plt.subplots(1, 2, figsize=(15, 7.5))

        # 왼쪽: 원본 이미지
        axes[0].imshow(original_image)
        axes[0].set_title(f"Original Image: {img_file}")
        axes[0].axis('off')

        # 오른쪽: 가상 정답 마스크가 겹쳐진 이미지
        axes[1].imshow(blended_image_np)
        axes[1].set_title(f"Segmentation ({num_classes - 1} labels)")
        axes[1].axis('off')

        plt.tight_layout()
        plt.show()

# ----------------------------------------------------
# 전체 파이프라인 실행 예시
# ----------------------------------------------------
image_dir = "/content/drive/MyDrive/2nd core sample dataset/raw image/cropped"
output_dir = "/content/drive/MyDrive/2nd core sample dataset/slic_sam_labels"

# 1단계: SLIC+SAM으로 가상 정답 생성 (이전에 실행한 경우 건너뛰기 가능)
create_pseudo_labels_with_slic(image_dir=image_dir, output_dir=output_dir)

# 2단계: 생성된 모든 이미지-레이블 쌍을 나란히 시각화
# visualize_all_side_by_side(image_dir=image_dir, pseudo_label_dir=output_dir)